In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [11]:
data =pd.read_excel("./data/horn_bill_telemetry.xlsx")

df = pd.DataFrame(data)

df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%y %H:%M')

# Extract month and year
df['month'] = df['timestamp'].dt.month
df['month_name'] = df['timestamp'].dt.strftime('%B')
df['year'] = df['timestamp'].dt.year

In [12]:
# Step 1: Filter the data for the specific bird
bird_name = '1_bill'  # Replace with the bird you want to track
bird_data = df[df['individual-local-identifier'] == bird_name].copy()

# Step 3: Time Series Analysis and Prediction with Time
# Convert timestamp to datetime format
bird_data['timestamp'] = pd.to_datetime(bird_data['timestamp'])

# Extract additional time-related features
bird_data['timestamp_ordinal'] = bird_data['timestamp'].apply(lambda x: x.toordinal())
bird_data['hour'] = bird_data['timestamp'].dt.hour
bird_data['minute'] = bird_data['timestamp'].dt.minute

# Prepare data for the model
X = bird_data[['timestamp_ordinal', 'hour', 'minute']]
y_long = bird_data['location-long']
y_lat = bird_data['location-lat']

# Split the data into train and test sets
X_train, X_test, y_long_train, y_long_test, y_lat_train, y_lat_test = train_test_split(X, y_long, y_lat, test_size=0.2, random_state=42)

# Train a linear regression model for longitude
model_long = LinearRegression()
model_long.fit(X_train, y_long_train)

# Train a linear regression model for latitude
model_lat = LinearRegression()
model_lat.fit(X_train, y_lat_train)

# Predict the location for a given date and time
given_datetime = pd.to_datetime('03-05-2015 11:30')  # Example date and time
given_datetime_features = pd.DataFrame({
    'timestamp_ordinal': [given_datetime.toordinal()],
    'hour': [given_datetime.hour],
    'minute': [given_datetime.minute]
})

predicted_long = model_long.predict(given_datetime_features)
predicted_lat = model_lat.predict(given_datetime_features)

print(f"Predicted Location on {given_datetime} - Longitude: {predicted_long[0]}, Latitude: {predicted_lat[0]}")

# Evaluate the model
long_mse = mean_squared_error(y_long_test, model_long.predict(X_test))
lat_mse = mean_squared_error(y_lat_test, model_lat.predict(X_test))

print(f"Longitude Prediction MSE: {long_mse}")
print(f"Latitude Prediction MSE: {lat_mse}")

Predicted Location on 2015-03-05 11:30:00 - Longitude: 92.9731179530747, Latitude: 26.93823172976417
Longitude Prediction MSE: 6.768704510894371e-06
Latitude Prediction MSE: 5.39389109891938e-06


In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict on the test set
y_long_pred = model_long.predict(X_test)
y_lat_pred = model_lat.predict(X_test)

# Evaluate Longitude predictions
long_mse = mean_squared_error(y_long_test, y_long_pred)
long_mae = mean_absolute_error(y_long_test, y_long_pred)
long_r2 = r2_score(y_long_test, y_long_pred)

# Evaluate Latitude predictions
lat_mse = mean_squared_error(y_lat_test, y_lat_pred)
lat_mae = mean_absolute_error(y_lat_test, y_lat_pred)
lat_r2 = r2_score(y_lat_test, y_lat_pred)

print(f"Longitude Prediction MSE: {long_mse}")
print(f"Longitude Prediction MAE: {long_mae}")
print(f"Longitude Prediction R²: {long_r2}")

print(f"Latitude Prediction MSE: {lat_mse}")
print(f"Latitude Prediction MAE: {lat_mae}")
print(f"Latitude Prediction R²: {lat_r2}")


Longitude Prediction MSE: 6.768704510894371e-06
Longitude Prediction MAE: 0.0019863987164547613
Longitude Prediction R²: 0.011878254411541112
Latitude Prediction MSE: 5.39389109891938e-06
Latitude Prediction MAE: 0.0014268339207661707
Latitude Prediction R²: 0.09574435077683774
